### Reading Data

**SPARK STREAMING**
-- we will read the data from the source to get the schema and assign it to (schema_entity) which we will use in streaming reading , It’s because Spark Structured Streaming refuses to "Infer Schema" on its own. 

The reasons why Spark forces this:
1-Stability: Streams are meant to run forever. If Spark "guessed" the schema every time a new file arrived, and one file had a slightly different format, the whole production pipeline would crash.

2-Performance: Inferring a schema requires Spark to read the file twice (once to guess types, once to load). In a high-speed stream, that's too slow.

3-The "Contract": By providing a schema, you are giving Spark a "contract." You are saying: "I guarantee the data will look like this."

In [0]:
entities=['customers','drivers','locations','payments','trips','vehicles']
for entity in entities:

        df_batch = spark.read.format('csv').option('header', True)\
        .option('inferSchema',True).\
        load(f"/Volumes/transportation_service_company/source/source_data/{entity}")


        schema_entity=df_batch.schema


        df = spark.readStream.format('csv') \
        .option('header', True).schema(schema_entity) \
        .load(f"/Volumes/transportation_service_company/source/source_data/{entity}")
         

        # Writing data to bronze layer and create a table on top of it
        df.writeStream.format('delta')\
        .outputMode('append')\
        .option('checkpointLocation',f'/Volumes/transportation_service_company/bronze/checkpoints/{entity}')\
        .trigger(once=True).toTable(f'transportation_service_company.bronze.{entity}')








In [0]:
import os 
curent=os.getcwd()
print(curent)

/Workspace/Users/majedabdu29@gmail.com/Transportation_service_company
